In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pathlib
from copy import copy
from matplotlib import cm, colors
import cv2

bins = [5,10,20,40]

baseline05 = 0.966
baseline025 = 0.95
baseline01 = 0.819

baseline = baseline05
dtresult = pd.read_csv('./Results/A/DT-0.5-os-3-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc','test-precision','test-recall']
    , header=1)
knnresult = pd.read_csv('./Results/A/KNN-0.5-os-3-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc','test-precision','test-recall']
    , header=1)
dtresult.insert(1, 'Classifier', 'DT')
knnresult.insert(1, 'Classifier', 'KNN')
masterdf = pd.concat([dtresult, knnresult])
masterdf.insert(1, "Threshold", 0.5)
masterdf = masterdf.sort_values(by=['cv-auc'], ascending=False)

####################
# Done with 0.5
####################

baseline = baseline025
dtresult = pd.read_csv('./Results/A/DT-0.25-os-3-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc','test-precision','test-recall']
    , header=1)
knnresult = pd.read_csv('./Results/A/KNN-0.25-os-3-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc','test-precision','test-recall']
    , header=1)
dtresult.insert(1, 'Classifier', 'DT')
knnresult.insert(1, 'Classifier', 'KNN')
dtresult.insert(1, 'Threshold', 0.25)
knnresult.insert(1, 'Threshold', 0.25)
masterdf = pd.concat([masterdf, knnresult, dtresult])
masterdf = masterdf.sort_values(by=['cv-auc'], ascending=False)

####################
# Done with 0.25
####################

baseline = baseline01
dtresult = pd.read_csv('./Results/A/DT-0.1-os-3-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc','test-precision','test-recall']
    , header=1)
knnresult = pd.read_csv('./Results/A/KNN-0.1-os-3-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc','test-precision','test-recall']
    , header=1)
dtresult.insert(1, 'Classifier', 'DT')
knnresult.insert(1, 'Classifier', 'KNN')
dtresult.insert(1, 'Threshold', 0.1)
knnresult.insert(1, 'Threshold', 0.1)
masterdf = pd.concat([masterdf, knnresult, dtresult])
masterdf = masterdf.sort_values(by=['cv-auc'], ascending=False)

####################
# Done with 0.10
####################

baselines = {0.1: 0.819, 0.25: 0.95, 0.5: 0.966}

print("   --- CV based on Z ---")
for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for Z in [1,3,5,7]:
            res = masterdf[(masterdf['Classifier'] == classifier) & (masterdf['Threshold'] == threshold) & (masterdf['windowSize'] == Z)]['test-auc'].iloc[0]
            print(f'{threshold} {classifier} {Z} {res} {res - baselines[threshold]}')

print("")
print("   --- CV based on XY ---")

for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for XY in [3,5,7]:
            res = masterdf[(masterdf['Classifier'] == classifier) & (masterdf['Threshold'] == threshold) & (masterdf['neighbourhood'] == XY)]['test-auc'].iloc[0]
            print(f'{threshold} {classifier} {XY} {res} {res - baselines[threshold]}')

print("")
print("   --- CV based on bins ---")

for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for bins in [5,10,20,40]:
            res = masterdf[(masterdf['Classifier'] == classifier) & (masterdf['Threshold'] == threshold) & (masterdf['buckets'] == bins)]['test-auc'].iloc[0]
            print(f'{threshold} {classifier} {bins} {res} {res - baselines[threshold]}')

print("")
print("   --- CV based on XYZ ---")

for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for Z in [1,3,5,7]:
            for XY in [3,5,7]:
                res = masterdf[(masterdf['Classifier'] == classifier) & (masterdf['Threshold'] == threshold) & (masterdf['windowSize'] == Z) & (masterdf['neighbourhood'] == XY)]['test-auc'].iloc[0]
                print(f'{threshold} {classifier} {Z} {XY} {res} {res - baselines[threshold]}')

columns = ['Threshold','Classifier', 'Type', 'Z', 'XY', 'Bins', 'ROC-AUC']
gooddf = pd.DataFrame(columns=columns)
for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for type in ['moran', 'scatter', 'spatstat']:
            for Z in [1,3,5,7]:
                for XY in [3,5,7]:
                    row = masterdf[(masterdf['Classifier'] == classifier)
                        & (masterdf['Threshold'] == threshold)
                        & (masterdf['type'] == type)
                        & (masterdf['windowSize'] == Z)
                        & (masterdf['neighbourhood'] == XY)].iloc[0]
                    nextRow =  pd.DataFrame([[threshold, classifier, type, Z, XY, row['buckets'], row['test-auc']]], columns=columns)
                    gooddf = pd.concat([gooddf, nextRow])

gooddf = gooddf.sort_values(by=['ROC-AUC'], ascending=False)


finaldf = pd.DataFrame(columns=columns)
for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for type in ['moran', 'scatter', 'spatstat']:
            row = gooddf[(gooddf['Classifier'] == classifier)
                & (gooddf['Threshold'] == threshold)
                & (gooddf['Type'] == type)].iloc[0]
            nextRow =  pd.DataFrame([[threshold, classifier, type, row['Z'], row['XY'], row['Bins'], row['ROC-AUC']]], columns=columns)
            finaldf = pd.concat([finaldf, nextRow])


   --- CV based on Z ---
0.1 DT 1 0.863095238095238 0.044095238095238076
0.1 DT 3 0.7589285714285715 -0.06007142857142844
0.1 DT 5 0.7827380952380953 -0.036261904761904606
0.1 DT 7 0.7529761904761905 -0.06602380952380948
0.1 KNN 1 0.7797619047619048 -0.03923809523809518
0.1 KNN 3 0.7529761904761905 -0.06602380952380948
0.1 KNN 5 0.8035714285714286 -0.015428571428571347
0.1 KNN 7 0.7797619047619048 -0.03923809523809518
0.25 DT 1 0.9005510364733667 -0.049448963526633305
0.25 DT 3 0.8821831540278142 -0.06781684597218574
0.25 DT 5 0.8017580687483601 -0.14824193125163987
0.25 DT 7 0.9130149567042772 -0.03698504329572272
0.25 KNN 1 0.8589609026502231 -0.09103909734977689
0.25 KNN 3 0.8357386512726319 -0.11426134872736804
0.25 KNN 5 0.8800839674626082 -0.06991603253739176
0.25 KNN 7 0.9043558121228024 -0.045644187877197595
0.5 DT 1 0.9236111111111112 -0.04238888888888881
0.5 DT 3 0.7789351851851852 -0.18706481481481474
0.5 DT 5 0.8726851851851851 -0.09331481481481485
0.5 DT 7 0.92361111111111